# System State in the Temperature-Entropy Diagram
To always have a well-defined temperature for the system we need *diagonal* states throughout the evolution.
This is guarantee by the dynamical map if the system starts from a diagonal state.

In [26]:
import glob
from IPython.display import display, Image
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
from qutip.visualization import plot_fock_distribution, plot_wigner

import tqdm

import uuid
import json
import os

from thermophaseonium.utilities.make_gif import create_gif
from thermophaseonium.utilities.states import Cavity, Ancilla

SEED = 42
rng = np.random.default_rng(SEED)

OMEGA = 0.5  # Interaction strength
N = 50  # Maximum number of dimensions

INTERACTION_TIME = 0.01
T_MAX = 1000
TIMESTEPS = np.linspace(0, T_MAX, int(T_MAX / INTERACTION_TIME))
print(f"Total of {len(TIMESTEPS)} timesteps from 0 to {T_MAX} with dt = {INTERACTION_TIME}")

Total of 100000 timesteps from 0 to 1000 with dt = 0.01


In [27]:
n_photons = 1
system = Cavity(qt.thermal_dm(N, n_photons), omega=1)

alpha = np.sqrt(0.4)
chi01_phase = np.pi * 0
chi02_phase = np.pi * 1
chi12_phase = np.pi * 0
chi01 = np.cos(chi01_phase) + 1j * np.sin(chi01_phase)
chi02 = np.cos(chi02_phase) + 1j * np.sin(chi02_phase)
chi12 = np.cos(chi12_phase) + 1j * np.sin(chi12_phase)

ancilla = Ancilla([
    [alpha**2, chi01, chi02],
    [chi01.conjugate(), (1 - alpha**2)/2, chi12],
    [chi02.conjugate(), chi12.conjugate(), (1 - alpha**2)/2]
])
print(ancilla.stable_temperature)
ancilla

2.059699076913792


Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[ 0.4+0.0000000e+00j  1. +0.0000000e+00j -1. +1.2246468e-16j]
 [ 1. +0.0000000e+00j  0.3+0.0000000e+00j  1. +0.0000000e+00j]
 [-1. -1.2246468e-16j  1. +0.0000000e+00j  0.3+0.0000000e+00j]]

## Evolution

In [28]:
def H_work(ancilla, omega=OMEGA):
    chi01 = ancilla.chi01
    chi02 = ancilla.chi02
    a = qt.destroy(N)
    upshift = (chi01 + chi02) * a.dag()
    return omega * (upshift + upshift.dag())

In [29]:
def D(op1, rho, op2=None):
    if op2 is None:
        op2 = op1
    return op1 * rho * op2.dag() - qt.commutator(op2.dag() * op1, rho, kind='anti') / 2

In [30]:
def onestep_evolution(rho, eta, dt=INTERACTION_TIME, o=OMEGA):
    work = 1j * o * dt * qt.commutator(H_work(eta), rho)
    heat1 = eta.ga * ( o * dt ) ** 2 * D(qt.create(N), rho)
    heat2 = eta.gb * ( o * dt ) ** 2 * D(qt.destroy(N), rho)
    rho_new = rho + work + heat1 + heat2
    return rho_new

In [31]:
def hilbert_is_good(rho, threshold=1e-6):
    """
    Check if the last element of the density matrix is almost null
    :param rho:
    :return:
    """
    return 0 <= abs(rho[-1, -1]) < threshold

In [32]:
def new_ancilla(ancilla, method='static', **kwargs):
    if method == 'static':
        return ancilla
    elif method == 'rotating':
        phis = np.linspace(0, 2 * np.pi, kwargs.get('timesteps', 10))
        ancilla.chi01 = phis[kwargs.get('step', -1)]
        ancilla.chi02 = phis[kwargs.get('step', -1)]
        return ancilla
    else:
        raise ValueError(f"Method {method} not recognized")


In [ ]:
rho = system
temperature_evolution = np.zeros(len(TIMESTEPS))
entropy_evolution = np.zeros(len(TIMESTEPS))
time_evolution = np.zeros(len(TIMESTEPS))

for k, t in enumerate(tqdm.tqdm(TIMESTEPS)):
    eta_k = ancilla
    rho_new = onestep_evolution(rho, eta_k)
    rho = Cavity(rho_new)
    time_evolution[k] = t
    try:
        temperature_evolution[k] = rho.temperature
        entropy_evolution[k] = rho.entropy
    except TypeError as e:
        print(f"Error in computing observables at step {k} with t = {t}: {e}")
        break
    if not hilbert_is_good(rho):
        print(f"Approximation no more valid at step {k} with t = {t}")
        break

rho = Cavity(rho)

 39%|███▉      | 39466/100000 [10:12<10:54, 92.45it/s]   

In [ ]:
def ancilla_text(ancilla_obj):
    return f"chi01: {ancilla_obj.chi01:.3f}\n" \
           f"chi02: {ancilla_obj.chi02:.3f}\n" \
           f"chi12: {ancilla_obj.chi12:.3f}"

In [ ]:
plt.plot(entropy_evolution, temperature_evolution)
plt.plot([entropy_evolution[0], entropy_evolution[-1]], [temperature_evolution[0], temperature_evolution[-1]], 'r')
plt.xlabel('Entropy')
plt.ylabel('Temperature')
plt.xlim(1.37, 1.7)
plt.ylim(1.41, 1.9)
plt.text(1.38, 1.75, ancilla_text(ancilla))

plt.show()